https://panel.holoviz.org/how_to/wasm/convert.html

https://www.openstreetmap.org/ => Search for elements

```python
#READ & lOAD YAML PARAMETERS
url = "https://raw.githubusercontent.com/thomleysens/Tutoriels_AME/main/tutoriels/game_params.yml"
r = requests.get(url)
params = yaml.safe_load(r.content)
RANGES = params["ranges"]
BUFFER_VALUE = params["buffer_value"]
NB_Q = params["nb_questions"]
CSV_SEP = ";"
CSV_FILE = "https://raw.githubusercontent.com/thomleysens/Tutoriels_AME/main/tutoriels/game_QA.csv"
WIKI_URL = params["url"]
```

```cmd
panel convert game_panel.ipynb --to pyodide-worker --out pyodide
```

```cmd
python3 -m http.server
```

http://localhost:8000/pyodide/game_panel.html